In [8]:
from torch.utils.data import Dataset
import torch
import pandas as pd
import json
import os
from pathlib import Path
import torch
import torch.nn as nn

class CropYieldDataset(Dataset):
    def __init__(self, data_lake_dir="../data/data_lake_organized", crop_name="corn", transform=None):
        self.samples = []
        self.transform = transform
        self.crop_name = crop_name.lower()
        
        fips_folders = [f for f in Path(data_lake_dir).iterdir() if f.is_dir()]
        
        for fips_folder in fips_folders:
            crop_json_path = fips_folder / f"{self.crop_name}.json"
            if not crop_json_path.exists():
                continue
            
            with open(crop_json_path, 'r') as f:
                yield_data = json.load(f)
            
            year_folders = [y for y in fips_folder.iterdir() if y.is_dir()]
            
            for year_folder in year_folders:
                year = year_folder.name
                weather_csv = year_folder / f"WeatherTimeSeries{year}.csv"
                
                if not weather_csv.exists():
                    continue
                if year not in yield_data:
                    continue
                
                df = pd.read_csv(weather_csv)
                
                # Only keep April–October
                df = df[(df['Month'] >= 4) & (df['Month'] <= 10)]

                # Drop non-weather columns
                df = df.drop(columns=['Year', 'Month', 'Day'], errors='ignore')

                # Make sure it's float tensor
                weather_tensor = torch.tensor(df.values, dtype=torch.float32)

                # Target
                yield_target = torch.tensor(yield_data[year]['yield'], dtype=torch.float32)

                self.samples.append((weather_tensor, yield_target))
    
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        x, y = self.samples[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

LSTM + TCN MODEL

In [9]:
class LSTMTCNRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, lstm_layers=1, tcn_channels=[64, 32]):
        super(LSTMTCNRegressor, self).__init__()
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=lstm_layers, batch_first=True)
        
        # TCN part: using 1D Convolutions
        self.tcn = nn.Sequential(
            nn.Conv1d(hidden_dim, tcn_channels[0], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(tcn_channels[0], tcn_channels[1], kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool1d(1)  # output shape: (batch, channels, 1)
        )
        
        self.fc = nn.Linear(tcn_channels[-1], 1)
    
    def forward(self, x):
        # x shape: (batch, time_steps, features)
        out, _ = self.lstm(x)  # (batch, time_steps, hidden_dim)
        out = out.permute(0, 2, 1)  # (batch, hidden_dim, time_steps) for Conv1D
        out = self.tcn(out)  # (batch, channels, 1)
        out = out.squeeze(-1)  # (batch, channels)
        out = self.fc(out)  # (batch, 1)
        return out.squeeze(-1)

Training Loop

In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim
import tqdm

def train_model(model, dataloader, num_epochs=30, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for epoch in tqdm.tqdm(range(num_epochs), desc="Training Progress"):
        epoch_loss = 0
        for x_batch, y_batch in dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        tqdm.tqdm.write(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss/len(dataloader):.4f}")
        tqdm.tqdm.set_description(f"Training Progress (Epoch {epoch+1} Loss: {epoch_loss/len(dataloader):.4f})")
    
    return model

In [17]:
dataset = CropYieldDataset(crop_name="corn")
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=lambda x: (nn.utils.rnn.pad_sequence([i[0] for i in x], batch_first=True), torch.stack([i[1] for i in x])))

# Example input_dim = number of weather features
input_dim = next(iter(dataloader))[0].shape[-1]

model = LSTMTCNRegressor(input_dim=input_dim)
trained_model = train_model(model, dataloader)

Training Progress: 100%|██████████| 30/30 [00:17<00:00,  1.72it/s]


Split the Dataset (Train/Validation)

In [18]:
from torch.utils.data import random_split

def get_dataloaders(dataset, train_ratio=0.8, batch_size=32):
    """
    Splits the dataset into training and validation sets.
    """
    total_len = len(dataset)
    train_len = int(total_len * train_ratio)
    val_len = total_len - train_len
    
    train_dataset, val_dataset = random_split(dataset, [train_len, val_len])

    def collate_fn(batch):
        xs = [b[0] for b in batch]
        ys = [b[1] for b in batch]
        xs = nn.utils.rnn.pad_sequence(xs, batch_first=True)
        ys = torch.stack(ys)
        return xs, ys

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    return train_loader, val_loader

Update training to track Validation Loss

In [ ]:
def train_model(model, train_loader, val_loader, num_epochs=30, lr=1e-3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    model.train()
    for epoch in tqdm.tqdm(range(num_epochs), desc="Training Progress"):
        epoch_loss = 0
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        # Validation loss
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(device), y_val.to(device)
                y_pred = model(x_val)
                val_loss += criterion(y_pred, y_val).item()
        
        """ print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss/len(train_loader):.4f} - Val Loss: {val_loss/len(val_loader):.4f}") """
    
    return model

Evaluate Final RMSE and MAE

In [22]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def evaluate_model(model, val_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            outputs = model(x_val)
            y_true.extend(y_val.numpy())
            y_pred.extend(outputs.cpu().numpy())
    
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mae = mean_absolute_error(y_true, y_pred)

    print(f"Final Validation RMSE: {rmse:.4f}")
    print(f" Final Validation MAE: {mae:.4f}")

    return rmse, mae

In [23]:
dataset = CropYieldDataset(crop_name="corn")
train_loader, val_loader = get_dataloaders(dataset)

# Detect input_dim
input_dim = next(iter(train_loader))[0].shape[-1]

model = LSTMTCNRegressor(input_dim=input_dim)
trained_model = train_model(model, train_loader, val_loader, num_epochs=30)

# Evaluate
evaluate_model(trained_model, val_loader)

Epoch 1/30 - Train Loss: 37258.3730 - Val Loss: 36691.2852
Epoch 2/30 - Train Loss: 35246.4209 - Val Loss: 32765.0942
Epoch 3/30 - Train Loss: 28416.3836 - Val Loss: 22040.7422
Epoch 4/30 - Train Loss: 15024.2370 - Val Loss: 6346.3361
Epoch 5/30 - Train Loss: 2125.1782 - Val Loss: 923.1875
Epoch 6/30 - Train Loss: 722.4004 - Val Loss: 409.1572
Epoch 7/30 - Train Loss: 418.0812 - Val Loss: 415.4073
Epoch 8/30 - Train Loss: 436.4736 - Val Loss: 396.2823
Epoch 9/30 - Train Loss: 418.1099 - Val Loss: 393.9786
Epoch 10/30 - Train Loss: 422.6292 - Val Loss: 393.6456
Epoch 11/30 - Train Loss: 426.8216 - Val Loss: 398.5365
Epoch 12/30 - Train Loss: 417.1492 - Val Loss: 392.7099
Epoch 13/30 - Train Loss: 415.6205 - Val Loss: 393.4336
Epoch 14/30 - Train Loss: 411.7903 - Val Loss: 391.4645
Epoch 15/30 - Train Loss: 415.7382 - Val Loss: 392.1481
Epoch 16/30 - Train Loss: 415.3316 - Val Loss: 391.6579
Epoch 17/30 - Train Loss: 416.4262 - Val Loss: 392.4838
Epoch 18/30 - Train Loss: 419.8716 - Val 

c:\Users\saadh\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


(19.385618, 15.784941)

| Strategy | Why it would help |
|:--|:--|
| Slightly increase LSTM hidden size (e.g., 128) | Capture more complex patterns |
| Add dropout (e.g., 0.2) between LSTM layers | Reduce overfitting further |
| Use a learning rate scheduler (cosine annealing) | Achieve more stable final convergence |
| Train for more epochs with early stopping | Allow better convergence and avoid overfitting |
| Normalize yield targets across counties | Handle variance across different yield scales |

Key hyperparameters to tune

| Hyperparameter | Values to try |
|:--|:--|
| LSTM hidden_dim | [64, 128, 256] |
| LSTM num_layers | [1, 2] |
| TCN channels | [[64, 32], [128, 64], [128, 128]] |
| Dropout | [0.0, 0.1, 0.2, 0.3] |
| Learning Rate | [1e-3, 5e-4, 1e-4] |

In [ ]:
import random
import numpy as np

def random_search_hyperparameters(
    dataset,
    num_trials=10,
    batch_size=32,
    num_epochs=30,
    device=None
):
    best_rmse = np.inf
    best_params = None
    best_model = None

    train_loader, val_loader = get_dataloaders(dataset, batch_size=batch_size)

    input_dim = next(iter(train_loader))[0].shape[-1]

    for trial in range(num_trials):
        # Randomly sample hyperparameters
        hidden_dim = random.choice([64, 128, 256])
        lstm_layers = random.choice([1, 2])
        tcn_channels = random.choice([[64, 32], [128, 64], [128, 128]])
        dropout_rate = random.choice([0.0, 0.1, 0.2, 0.3])
        lr = random.choice([1e-3, 5e-4, 1e-4])

        print(f"\nTrial {trial+1}/{num_trials}")
        print(f"LSTM hidden_dim: {hidden_dim}, LSTM layers: {lstm_layers}, TCN channels: {tcn_channels}, Dropout: {dropout_rate}, LR: {lr}")

        model = LSTMTCNRegressor(
            input_dim=input_dim,
            hidden_dim=hidden_dim,
            lstm_layers=lstm_layers,
            tcn_channels=tcn_channels
        )

        trained_model = train_model(model, train_loader, val_loader, num_epochs=num_epochs, lr=lr)
        
        # Evaluate
        rmse, mae = evaluate_model(trained_model, val_loader)

        # Save if best
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = {
                'hidden_dim': hidden_dim,
                'lstm_layers': lstm_layers,
                'tcn_channels': tcn_channels,
                'dropout': dropout_rate,
                'lr': lr
            }
            best_model = trained_model

    print("\n🏆 Best RMSE:", best_rmse)
    print("🏆 Best Hyperparameters:", best_params)

    return best_model, best_params

In [25]:
dataset = CropYieldDataset(crop_name="corn")
best_model, best_params = random_search_hyperparameters(dataset, num_trials=10, batch_size=32, num_epochs=30)


🚀 Trial 1/10
LSTM hidden_dim: 64, LSTM layers: 1, TCN channels: [64, 32], Dropout: 0.3, LR: 0.0001
Epoch 1/30 - Train Loss: 37299.2126 - Val Loss: 39135.4443
Epoch 2/30 - Train Loss: 37331.3901 - Val Loss: 39098.7939
Epoch 3/30 - Train Loss: 37287.8717 - Val Loss: 39056.4912
Epoch 4/30 - Train Loss: 37246.8058 - Val Loss: 38997.8965
Epoch 5/30 - Train Loss: 37168.7048 - Val Loss: 38909.3008
Epoch 6/30 - Train Loss: 37037.1682 - Val Loss: 38755.4160
Epoch 7/30 - Train Loss: 36862.4953 - Val Loss: 38552.6826
Epoch 8/30 - Train Loss: 36616.4322 - Val Loss: 38296.3662
Epoch 9/30 - Train Loss: 36312.5898 - Val Loss: 37938.0693
Epoch 10/30 - Train Loss: 35925.3125 - Val Loss: 37486.1826
Epoch 11/30 - Train Loss: 35453.0767 - Val Loss: 36893.2393
Epoch 12/30 - Train Loss: 34834.1102 - Val Loss: 36095.1611
Epoch 13/30 - Train Loss: 33882.4431 - Val Loss: 35128.1338
Epoch 14/30 - Train Loss: 32932.3531 - Val Loss: 34030.8789
Epoch 15/30 - Train Loss: 31744.6339 - Val Loss: 32605.0835
Epoch 16/

c:\Users\saadh\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/30 - Train Loss: 36938.1696 - Val Loss: 38223.0186
Epoch 2/30 - Train Loss: 35314.3164 - Val Loss: 35172.3057
Epoch 3/30 - Train Loss: 30299.4941 - Val Loss: 27021.6357
Epoch 4/30 - Train Loss: 19213.6053 - Val Loss: 12054.4785
Epoch 5/30 - Train Loss: 4707.0665 - Val Loss: 576.8844
Epoch 6/30 - Train Loss: 713.6747 - Val Loss: 597.1596
Epoch 7/30 - Train Loss: 583.5525 - Val Loss: 539.7526
Epoch 8/30 - Train Loss: 449.7409 - Val Loss: 394.4332
Epoch 9/30 - Train Loss: 435.0487 - Val Loss: 385.7860
Epoch 10/30 - Train Loss: 434.5342 - Val Loss: 405.0880
Epoch 11/30 - Train Loss: 427.1166 - Val Loss: 400.5533
Epoch 12/30 - Train Loss: 429.5379 - Val Loss: 396.4790
Epoch 13/30 - Train Loss: 430.0982 - Val Loss: 406.6424
Epoch 14/30 - Train Loss: 427.0948 - Val Loss: 397.0959
Epoch 15/30 - Train Loss: 427.5071 - Val Loss: 395.3900
Epoch 16/30 - Train Loss: 430.9818 - Val Loss: 402.0762
Epoch 17/30 - Train Loss: 426.5639 - Val Loss: 387.5298
Epoch 18/30 - Train Loss: 423.3556 - Val

c:\Users\saadh\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/30 - Train Loss: 37263.0734 - Val Loss: 39040.8584
Epoch 2/30 - Train Loss: 37146.4138 - Val Loss: 38844.4482
Epoch 3/30 - Train Loss: 36830.1752 - Val Loss: 38158.7832
Epoch 4/30 - Train Loss: 35590.5586 - Val Loss: 36040.6650
Epoch 5/30 - Train Loss: 32508.5319 - Val Loss: 31638.0874
Epoch 6/30 - Train Loss: 27175.3192 - Val Loss: 24955.1343
Epoch 7/30 - Train Loss: 19902.2593 - Val Loss: 16762.6089
Epoch 8/30 - Train Loss: 11937.8451 - Val Loss: 8784.5266
Epoch 9/30 - Train Loss: 5300.2380 - Val Loss: 3229.2420
Epoch 10/30 - Train Loss: 1584.4997 - Val Loss: 812.8403
Epoch 11/30 - Train Loss: 483.6343 - Val Loss: 389.0194
Epoch 12/30 - Train Loss: 450.2479 - Val Loss: 379.0124
Epoch 13/30 - Train Loss: 444.7738 - Val Loss: 388.4583
Epoch 14/30 - Train Loss: 435.8608 - Val Loss: 397.9139
Epoch 15/30 - Train Loss: 427.3928 - Val Loss: 410.4777
Epoch 16/30 - Train Loss: 429.7228 - Val Loss: 404.5256
Epoch 17/30 - Train Loss: 426.1556 - Val Loss: 402.9701
Epoch 18/30 - Train Los

...

c:\Users\saadh\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Final Validation RMSE: 19.9459
 Final Validation MAE: 16.1970

🚀 Trial 9/10
LSTM hidden_dim: 64, LSTM layers: 1, TCN channels: [128, 128], Dropout: 0.1, LR: 0.001
Epoch 1/30 - Train Loss: 36357.7444 - Val Loss: 35402.3203
Epoch 2/30 - Train Loss: 25889.9441 - Val Loss: 13562.0850
Epoch 3/30 - Train Loss: 4034.5850 - Val Loss: 2335.2167
Epoch 4/30 - Train Loss: 1139.9075 - Val Loss: 1017.9266
Epoch 5/30 - Train Loss: 626.8055 - Val Loss: 377.4245
Epoch 6/30 - Train Loss: 473.8778 - Val Loss: 430.4884
Epoch 7/30 - Train Loss: 443.9705 - Val Loss: 400.0341
Epoch 8/30 - Train Loss: 439.4020 - Val Loss: 395.4657
Epoch 9/30 - Train Loss: 432.9635 - Val Loss: 407.9025
Epoch 10/30 - Train Loss: 439.7967 - Val Loss: 407.0040
Epoch 11/30 - Train Loss: 436.4842 - Val Loss: 391.1179
Epoch 12/30 - Train Loss: 435.2982 - Val Loss: 406.6898
Epoch 13/30 - Train Loss: 442.6236 - Val Loss: 383.7454
Epoch 14/30 - Train Loss: 437.3283 - Val Loss: 392.7545
Epoch 15/30 - Train Loss: 431.4270 - Val Loss: 421

c:\Users\saadh\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Epoch 1/30 - Train Loss: 36578.9023 - Val Loss: 35571.7480
Epoch 2/30 - Train Loss: 25941.2315 - Val Loss: 14755.4985
Epoch 3/30 - Train Loss: 4791.0238 - Val Loss: 1026.3358
Epoch 4/30 - Train Loss: 1493.5519 - Val Loss: 406.0776
Epoch 5/30 - Train Loss: 604.6167 - Val Loss: 570.5349
Epoch 6/30 - Train Loss: 470.3337 - Val Loss: 381.7330
Epoch 7/30 - Train Loss: 432.2075 - Val Loss: 439.4921
Epoch 8/30 - Train Loss: 433.9936 - Val Loss: 398.6534
Epoch 9/30 - Train Loss: 431.1563 - Val Loss: 398.7602
Epoch 10/30 - Train Loss: 429.3092 - Val Loss: 408.0958
Epoch 11/30 - Train Loss: 432.5579 - Val Loss: 400.2274
Epoch 12/30 - Train Loss: 431.7738 - Val Loss: 403.4737
Epoch 13/30 - Train Loss: 431.0677 - Val Loss: 408.8828
Epoch 14/30 - Train Loss: 433.7791 - Val Loss: 393.0583
Epoch 15/30 - Train Loss: 430.0462 - Val Loss: 422.6017
Epoch 16/30 - Train Loss: 428.8135 - Val Loss: 391.9586
Epoch 17/30 - Train Loss: 429.5713 - Val Loss: 410.1427
Epoch 18/30 - Train Loss: 437.8155 - Val Loss:

c:\Users\saadh\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
